In [2]:
!pip install gensim==3.8.1
from gensim.models import KeyedVectors

path_model = "embeddings-l-model.vec"
# Load vectors directly from the file
modelWV = KeyedVectors.load_word2vec_format(path_model, limit=300000)
DIMENSION=300

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
from datasets import load_dataset, DatasetDict

raw_dataset = load_dataset("javilonso/mex_data", use_auth_token=True)

Using custom data configuration javilonso--mex_data-0c030e0c8e30da9e
Reusing dataset parquet (/home/javilonso/.cache/huggingface/datasets/parquet/javilonso--mex_data-0c030e0c8e30da9e/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
# Extract data
X_train = raw_dataset['train']['Opinion']
y_train_attraction = raw_dataset['train']['Attraction']
y_train_polarity = raw_dataset['train']['Polarity']

X_test = raw_dataset['test']['Opinion']
y_test_attraction = raw_dataset['test']['Attraction']
y_test_polarity = raw_dataset['test']['Polarity']


In [8]:
import numpy as np

def getDocVec(text):
    """gets a text and returns a vector for this text"""

    tokens=text.split()
    data=[]
    n=0
    for token in tokens:
        token=token.lower()
        if token in modelWV.vocab:
            data.append(modelWV[token])
            n += 1
            
    if len(data)>0:     
        #vectors = np.array(data)
        res = [sum(i)/n for i in zip(*data)]
    else:
        res =[0]*DIMENSION

    #return np.average(vectors, axis=0)
    return res

# text="Esto es un ejemplo de texto"
# print(len(getDocVec(text)))

In [9]:
X_vectors_train = []
for text in X_train:
  docVector = getDocVec(text)
  X_vectors_train.append(docVector)
  
print('text vectors for training were created' )

text vectors for training were created


In [10]:
X_vectors_test = []
for text in X_test:
  docVector=getDocVec(text)
  X_vectors_test.append(docVector)
  
print('text vectors for test were created' )

text vectors for test were created


In [11]:
print(len(X_vectors_train), len(y_train_attraction))
print(len(X_vectors_train[0]))
print(len(X_vectors_test), len(y_test_attraction))


23944 23944
300
5987 5987


In [12]:
from sklearn import svm

modelSVM=svm.SVC()
modelSVM.fit(X_vectors_train, y_train_attraction)
print('model was trained!!')

model was trained!!


In [13]:
# TEST Predictions
from sklearn.metrics import classification_report

y_pred=modelSVM.predict(X_vectors_test)
print(classification_report(y_test_attraction, y_pred))


              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3265
           1       0.93      0.91      0.92      1667
           2       0.97      0.97      0.97      1055

    accuracy                           0.95      5987
   macro avg       0.95      0.95      0.95      5987
weighted avg       0.95      0.95      0.95      5987



In [16]:
from sklearn import svm

modelSVM=svm.SVC()
modelSVM.fit(X_vectors_train,y_train_polarity)
print('model was trained!!')

model was trained!!


In [17]:
# TEST Predictions
from sklearn.metrics import classification_report

y_pred=modelSVM.predict(X_vectors_test)
print(classification_report(y_test_polarity, y_pred))


              precision    recall  f1-score   support

           0       0.40      0.02      0.04       100
           1       0.00      0.00      0.00       140
           2       0.41      0.18      0.25       408
           3       0.00      0.00      0.00      1148
           4       0.72      1.00      0.84      4191

    accuracy                           0.71      5987
   macro avg       0.31      0.24      0.22      5987
weighted avg       0.54      0.71      0.60      5987



/home/javilonso/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/javilonso/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/javilonso/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
